In [32]:
from google.colab import files
uploaded = files.upload()

Saving 1unb_bin.csv to 1unb_bin (1).csv


In [52]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Load the dataset
url_dataset = pd.read_csv('1unb_bin.csv')  # Replace 'url_dataset.csv' with your actual dataset file name

# Separate the features and labels
X = url_dataset.iloc[:, 2:]  # Select all columns except the last one
y = url_dataset['Label']  # Select the last column as the target variable

# Encode the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# Train the models
rf_model = RandomForestClassifier(n_estimators=200, criterion='gini', min_samples_split=2, max_depth=100, random_state=42)
rf_model.fit(X_train, y_train)


dt_model = DecisionTreeClassifier(max_depth=100, criterion='gini', splitter='best')
dt_model.fit(X_train, y_train)

# Create the neural network model
model = Sequential()
model.add(Dense(20, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, pd.get_dummies(y_train), epochs=500, batch_size=512, verbose=1)



Epoch 1/500
227/227 [==============================] - 1s 2ms/step - loss: 0.3990 - accuracy: 0.8433
Epoch 2/500
227/227 [==============================] - 0s 2ms/step - loss: 0.2711 - accuracy: 0.9028
Epoch 3/500
227/227 [==============================] - 0s 2ms/step - loss: 0.2343 - accuracy: 0.9077
Epoch 4/500
227/227 [==============================] - 0s 2ms/step - loss: 0.2103 - accuracy: 0.9166
Epoch 5/500
227/227 [==============================] - 0s 2ms/step - loss: 0.1916 - accuracy: 0.9230
Epoch 6/500
227/227 [==============================] - 0s 2ms/step - loss: 0.1782 - accuracy: 0.9274
Epoch 7/500
227/227 [==============================] - 0s 2ms/step - loss: 0.1681 - accuracy: 0.9332
Epoch 8/500
227/227 [==============================] - 0s 2ms/step - loss: 0.1596 - accuracy: 0.9384
Epoch 9/500
227/227 [==============================] - 0s 2ms/step - loss: 0.1526 - accuracy: 0.9423
Epoch 10/500
227/227 [==============================] - 1s 2ms/step - loss: 0.1467 - accura

In [53]:
# Make predictions on the test data
rf_predictions = rf_model.predict(X_test)
dt_predictions = dt_model.predict(X_test)
nn_predictions = model.predict(X_test)
nn_predictions_classes = np.argmax(nn_predictions, axis=1)

# Create the ensemble prediction using majority voting
ensemble_predictions = []
for rf_pred, dt_pred, nn_pred in zip(rf_predictions, dt_predictions, nn_predictions_classes):
    votes = [rf_pred, dt_pred, nn_pred]
    majority_vote = max(set(votes), key=votes.count)
    ensemble_predictions.append(majority_vote)

# Decode the predictions
ensemble_predictions_decoded = label_encoder.inverse_transform(ensemble_predictions)

# Decode the true labels
y_test_decoded = label_encoder.inverse_transform(y_test)

# Evaluate the ensemble model
ensemble_accuracy = accuracy_score(y_test_decoded, ensemble_predictions_decoded)
print("Ensemble Accuracy:", ensemble_accuracy)

# Print classification report with 5 decimal places
ensemble_report = classification_report(y_test_decoded, ensemble_predictions_decoded, digits=5)
print("Ensemble Classification Report:")
print(ensemble_report)

1551/1551 [==============================] - 2s 1ms/step
Ensemble Accuracy: 0.9973594033460996
Ensemble Classification Report:
              precision    recall  f1-score   support

      Benign    0.99139   0.99633   0.99385     10631
   Malicious    0.99900   0.99764   0.99832     38979

    accuracy                        0.99736     49610
   macro avg    0.99519   0.99699   0.99609     49610
weighted avg    0.99737   0.99736   0.99736     49610

